In [4]:
# Libraries
from TurkishStemmer import TurkishStemmer
from operator import itemgetter

import pandas as pd
import gensim
import pprint
import string
import spacy
import re

In [5]:
NUMBERS = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 9999
spacy_nlp = spacy.blank('tr')
stop_words = list(spacy_nlp.Defaults.stop_words)
pp = pprint.PrettyPrinter(indent=4)
punctuations = string.punctuation
stemmer = TurkishStemmer()

dropped = [
 'TEKSTIL_KOL_TIPI',
 'TEKSTIL_KOL_TIPI_TANIMI',
 'TEKSTIL_PACA_TIPI',
 'TEKSTIL_PACA_TIPI_TANIMI',
 'TEKSTIL_YAKA_TIPI',
 'TEKSTIL_YAKA_TIPI_TANIMI',
 'WEB_KISA_ACIKLAMA',
 'META_KEY_ACIKLAMA',
 'TEKNIK_SERVIS_ACIKLAMA',
 'EN_MALZEME_UZUN_ACIKLAMA',
 'REYON_TANIM',
 'TEKSTIL_KOL_TIPI',
 'TEKSTIL_YAKA_TIPI',
 'WEB_UZUN_ACIKLAMA',
 'WEB_KISA_ACIKLAMA',
 'META_KEY_ACIKLAMA',
 'TEKNIK_SERVIS_ACIKLAMA'
 ]

In [6]:
def filter(query):
  # Remove puncuations
  query = re.sub(r'[^\w\s]',' ',query)

  # Remove extra spaces
  query = re.sub(' +',' ',query)

  # Get tokens
  tokens = spacy_nlp(query)

  # Lower, strip and lemmatize
  tokens = [word.text.lower().strip() for word in tokens]


  # Remove stopwords, and exclude words less than 2 characters
  tokens = [ word for word in tokens if 
            (word not in stop_words) and 
            (word not in punctuations) and 
            (word in NUMBERS or len(word) >= 2) and 
            (word != "yok") and
            (word != "nan")]

  # Stem tokens - Does not works well :(
  # tokens = [stemmer.stem(word) for word in tokens]

  return tokens

In [43]:
def search_similar_products(search_term, num_best=100):
    query_bow = dictionary.doc2bow(filter(search_term))
    query_tfidf = ebebek_tfidf_model[query_bow]
    query_lsi = ebebek_lsi_model[query_tfidf]

    malzeme_index.num_best = num_best

    malzeme_list = malzeme_index[query_lsi]

    malzeme_list.sort(key=itemgetter(1), reverse=True)
    malzeme_names = []

    for j, malzeme in enumerate(malzeme_list):

        malzeme_names.append (
            {
                'SEARCH_TERM': search_term,
                'STOK_ADI': df['STOK_ADI'][malzeme[0]],
                'MALZEME_UZUN_ACIKLAMA': df['MALZEME_UZUN_ACIKLAMA'][malzeme[0]],
                'RELEVANCE': round((malzeme[1] * 100),2)
            }

        )
        if j == (malzeme_index.num_best-1):
            break

    return pd.DataFrame(malzeme_names, columns=['SEARCH_TERM', 'STOK_ADI', 'MALZEME_UZUN_ACIKLAMA', 'RELEVANCE'])


In [8]:
# Read data and filter df
df = pd.read_excel("./dataset/ebebek.xlsx")
df.drop(columns=dropped, axis=1, inplace=True)
df

,STOK_ADI,ANA_KATEGORI_ADI,WEB_ANA_KATEGORI_TANIMI,KATEGORI_ADI,ALT_KATEGORI_ADI,MARKA_ADI,RENK_ADI,BEDEN,TEKSTIL_URUN_GRUBU_ADI,TEKSTIL_CINSIYET,TEKSTIL_ADET_ADI,MALZEME_UZUN_ACIKLAMA,ASKI_DURUMU,EN_STOK_ADI,ANALIZ_KATEGORISI,WEB_BIRINCI_KATEGORI_TANIMI,WEB_IKINCI_KATEGORI_TANIMI,WEB_UCUNCU_KATEGORI_TANIMI,MAGAZA_KONUMU
0,Hazır Büyük Boy Hediye Paketi,Sarf Malzemeler,NaN,NaN,NaN,ebebek,NaN,NaN,NaN,NaN,Tek,Hazır Büyük Boy Hediye Paketi,NaN,NaN,NaN,Sarf Malzemeler,NaN,NaN,Masa
1,075MM X 040MM ECO TERMAL BASKISIZ ETİKET,Sarf Malzemeler,Sarf Malzemeler,Sarf Malzemeler,Sarf Malzemeler,Bebek,NaN,NaN,NaN,NaN,Tek,075MM X 040MM ECO TERMAL BASKISIZ ETİKET,NaN,Şeffaf Ebebek Baskılı 75x100 cm,NaN,Sarf Malzemeler,Sarf Malzemeler,Sarf Malzemeler,Diğer
2,075MM X 075MM ECO TERMAL BASKISIZ ETİKET,Sarf Malzemeler,Sarf Malzemeler,Sarf Malzemeler,Sarf Malzemeler,Bebek,NaN,NaN,NaN,NaN,Tek,075MM X 075MM ECO TERMAL BASKISIZ ETİKET,NaN,Şeffaf Ebebek Baskılı 75x100 cm,NaN,Sarf Malzemeler,Sarf Malzemeler,Sarf Malzemeler,Diğer
3,"(OUT)Graco Slim Spaces Otomatik Salınc,",Bebek Araç Gereç ve Mobilya,Bebek Salıncakları,Ana Kucakları ve Salıncaklar,Pilli,Outlet,Linus,NaN,NaN,1046.0,Tek,(OUT) Graco Slim Spaces Otomatik Salıncak,NaN,"(OUT)Graco Slim Spaces Otomatic Swi, Lin",Ana Kucakları ve Salıncaklar,Araç & Gereç,Ana Kucağı,Bebek Salıncakları,Diğer
4,(OUT)baby plus Tek Vantuzlu Güneşlik 2 A,Güvenlik Ürünleri,Bebek Araba Güneşliği,Otomobilde Güvenlik Ürünleri,Güneşlikler,Outlet,NaN,NaN,NaN,NaN,Tek,(OUT)baby plus Car Sun Shade with Single Sucti...,NaN,(OUT)Window Shield (Set of 2),Otomobilde Güvenlik Ürünleri - Güneşlikler,Güvenlik,Araba için Güvenlik Ürünleri,Bebek Araba Güneşliği,Diğer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226258,"Mushi Pirates Jean Şort Ta, Siyah, 4 Yaş",Tekstil,Bebek Alt Üst Takım,Örme,Bebek Giyim,Mushi,Siyah,4 Yaş,Patiksiz Alt,1044.0,2li,Mushi Pirates Jean Şort Takım,Askı,NaN,Patiksiz Alt,Giyim & Tekstil,Bebek Giyim,Bebek Alt Üst Takım,Bebek Giyim
226259,"Mushi Hush Hip-Hop Baggy 3, Yeşil, 3 Yaş",Tekstil,Bebek Alt Üst Takım,Örme,Bebek Giyim,Mushi,Yeşil,3 Yaş,Patiksiz Alt,1044.0,2li,Mushi Hush Hip-Hop Baggy 3Lü Takım,Askı,NaN,Patiksiz Alt,Giyim & Tekstil,Bebek Giyim,Bebek Alt Üst Takım,Bebek Giyim
226260,"Mushi Hush Hip-Hop Baggy 3, Yeşil, 4 Yaş",Tekstil,Bebek Alt Üst Takım,Örme,Bebek Giyim,Mushi,Yeşil,4 Yaş,Patiksiz Alt,1044.0,2li,Mushi Hush Hip-Hop Baggy 3Lü Takım,Askı,NaN,Patiksiz Alt,Giyim & Tekstil,Bebek Giyim,Bebek Alt Üst Takım,Bebek Giyim
226261,"Mushi Like Comics Şort Tak, Siyah, 2 Yaş",Tekstil,Bebek Alt Üst Takım,Örme,Bebek Giyim,Mushi,Siyah,2 Yaş,Patiksiz Alt,1044.0,2li,Mushi Like Comics Şort Takım,Askı,NaN,Patiksiz Alt,Giyim & Tekstil,Bebek Giyim,Bebek Alt Üst Takım,Bebek Giyim


In [9]:
dictionary = gensim.corpora.Dictionary.load("./models/dictionary")
malzeme_index = gensim.similarities.MatrixSimilarity.load("./models/malzeme_index")

In [10]:
# Load TFIDF and LSI models
ebebek_tfidf_model = gensim.models.TfidfModel.load("./models/tfidf/ebebek_tfidf")
ebebek_lsi_model = gensim.models.LsiModel.load("./models/lsi/ebebek_lsi")

In [50]:
search_df = pd.read_excel("./dataset/ekim_kasim_verisi.xlsx", "Veri kümesi1")
search_terms = search_df["Arama Terimi"].values[:100]
search_terms

/Users/seyyidosmansevgili/Library/Python/3.9/lib/python/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


array(['Mont', 'Uyku tulumu', 'Tulum', 'ahşap beşik', 'Mama sandalyesi',
       'Bebek bezi', 'Emzik', 'Biberon', 'Park yatak', 'Zıbın', 'Termos',
       '0-13 kg', 'Hastane çıkışı', 'goon külot bez', 'Ana kucağı',
       'bear hugs', 'Oioi', 'animals party', 'Islak mendil', 'Mama',
       'Aptamil', 'Yelek', 'Sleepy', 'Elbise', 'Bebek mont', 'Hipp',
       'Battaniye', 'Yürüteç', 'Kanguru', 'flowers girl', 'Çanta', 'Sma',
       'Atlet', 'reflü yatağı', 'Oyun halısı', 'Ateş ölçer',
       'Akülü araba', '[teks-til]', 'uyku tulumu', 'Bez', 'Bebek giyim',
       'Yatak', 'Oto koltuğu', 'babyz ayıcık', 'biberon', 'Bisiklet',
       'Yenidoğan', 'Panduf', 'Mustela', 'Bebek tulum', 'polygreen',
       'friendly', 'mont', 'Fisher price', 'Avent', 'Erkek bebek',
       'İlk arabam', 'Molfix', 'Body', 'Yeni doğan', 'Lansinoh', 'Dalin',
       'Yenidoğan giyim', 'Bere', 'Patik', 'Baby me', 'Şampuan',
       'Ayakkabı', 'tulum', 'Müslin', 'Araba', 'Hamilton', 'Salıncak',
       'Erkek bebek mon

In [49]:
result_df = pd.DataFrame()
for search_term in search_terms:
  result_df = pd.concat([result_df,search_similar_products(search_term, 30)])
result_df.to_excel("./results.xlsx")

* kız bebek elbise kırmızı
* erkek oyuncak
* zeka oyunları
* güneş kremi

In [12]:
search_term = "zeka"
query_bow = dictionary.doc2bow(filter(search_term))
query_tfidf = ebebek_tfidf_model[query_bow]
query_lsi = ebebek_lsi_model[query_tfidf]

malzeme_index.num_best = 20

malzeme_list = malzeme_index[query_lsi]

malzeme_list.sort(key=itemgetter(1), reverse=True)
malzeme_names = []

for j, malzeme in enumerate(malzeme_list):

    malzeme_names.append (
        {
            'Relevance': round((malzeme[1] * 100),2),
            'STOK_ADI': df['STOK_ADI'][malzeme[0]],
            'MALZEME_UZUN_ACIKLAMA': df['MALZEME_UZUN_ACIKLAMA'][malzeme[0]]
        }

    )
    if j == (malzeme_index.num_best-1):
        break
pd.DataFrame(malzeme_names, columns=['Relevance', 'STOK_ADI', 'MALZEME_UZUN_ACIKLAMA'])

,Relevance,STOK_ADI,MALZEME_UZUN_ACIKLAMA
0,67.10,Mikado Zeka Geliştirici Oyunlar,Mikado Zeka Geliştirici Oyunlar
1,63.22,Mikado Sayılar 1-2-3 Flash Kart,Mikado Sayılar 1-2-3 Flash Kart
2,63.16,Mikado Öfke Nöbeti,Mikado Öfke Nöbeti
3,62.56,Mikado Dokun Hisset Kartlarım,Mikado Dokun Hisset Kartlarım IH
4,61.89,Mikado Oyuncaklar Flash Kart,Mikado Oyuncaklar Flash Kart
5,61.63,Mikado Hayvan Çizilen Kartlar Flash Kart,Mikado Hayvan Çizilen Kartlar Flash Kart
6,60.45,Mikado Bebeğimin İlk Kartları,Mikado Bebeğimin İlk Kartları IH
7,59.84,Mikado İlk Kelimelerim,Mikado İlk Kelimelerim IH
8,59.04,"Mikado Adım Adım Bebek Gelişim Se, 14 Ay",NaN
9,58.85,"Mikado Adım Adım Bebek Gelişim Se, 33 Ay",NaN


In [13]:
query_lsi

[(0, 2.2872313137940193e-05),
 (1, 6.780204274523666e-05),
 (2, -3.47153719072918e-05),
 (3, 0.00032573957902715404),
 (4, 5.333692711191212e-06),
 (5, 8.838905191347539e-05),
 (6, -0.00022499577692436528),
 (7, 0.001191798268540586),
 (8, -0.00023833732800798368),
 (9, 0.0023980204551244254),
 (10, 0.001942548333444824),
 (11, -0.0012855152763011919),
 (12, 3.403758830986943e-05),
 (13, 0.00031398355838504045),
 (14, -0.0010850472214362081),
 (15, -0.0009965222868301726),
 (16, -0.00028626384208177045),
 (17, 0.0004693353248059827),
 (18, 0.0020239599140334753),
 (19, 0.00014520498713487784),
 (20, -0.00045706740994259026),
 (21, -0.00040126418899978297),
 (22, 5.841954489488991e-05),
 (23, 0.00034180926118162114),
 (24, 0.00027806930589282776),
 (25, 0.00026586884060290623),
 (26, 2.414656051234984e-05),
 (27, 1.5886676563658084e-05),
 (28, -7.465958158847584e-05),
 (29, -0.00021178131748235212),
 (30, 1.0991777047652819e-05),
 (31, -0.0004049920557261532),
 (32, -1.1473883492676606e

In [14]:
query_tfidf

[(2580, 0.835509960534216)]

In [15]:
malzeme_list

[(116175, 0.670975387096405),
 (208103, 0.6321749091148376),
 (110234, 0.6316291093826294),
 (222872, 0.6256201267242432),
 (116176, 0.6189289689064026),
 (143498, 0.6163418889045715),
 (116164, 0.6045081615447998),
 (208101, 0.5984023809432983),
 (192733, 0.5904217958450317),
 (219567, 0.5885381102561951),
 (119012, 0.587088942527771),
 (119010, 0.5867092609405518),
 (119011, 0.5857290029525757),
 (145625, 0.5853299498558044),
 (119003, 0.5850870609283447),
 (216160, 0.5821094512939453),
 (190025, 0.5760336518287659),
 (219566, 0.5739709734916687),
 (120248, 0.573230504989624),
 (219565, 0.5726778507232666)]